Paraphrase is all about the writing same sentence with different words.

In [1]:
!pip install sentence-splitter
!pip install transformers
!pip install sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.7 MB/s eta 0:00:00


Downloading pre trained Google Pegasus paraphrase model and it's tokenizer

In [2]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

In [3]:
model=PegasusForConditionalGeneration.from_pretrained("tuner007/pegasus_paraphrase") # code 007 we use from huggingface
tokenizer=PegasusTokenizer.from_pretrained("tuner007/pegasus_paraphrase")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at tuner007/pegasus_paraphrase and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Tokenization

In [4]:
text='The ultimate test of your knowledge is your capacity to adapt to change.'

batch = tokenizer([text], padding =True, truncation=True, max_length=60, return_tensors='pt')

output=model.generate(**batch, max_length=60, num_beams=5, num_return_sequences=5, temperature=1.5)

results=tokenizer.batch_decode(output, skip_special_tokens=True)
results

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2691: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Keyword arguments {'trauncation': True} not recognized.
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


['Your capacity to adapt to change is the ultimate test of your knowledge.',
 'Your ability to adapt to change is the ultimate test of your knowledge.',
 'The ability to adapt to change is the ultimate test of your knowledge.',
 'Your capacity to adapt to change is the test of your knowledge.',
 'The test of your knowledge is your ability to adapt to change.']

Save model and tokenizer

In [5]:
model.save_pretrained('pegasus_paraphrase')
tokenizer.save_pretrained('pegasus_paraphrase')

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 60, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


('pegasus_paraphrase/tokenizer_config.json',
 'pegasus_paraphrase/special_tokens_map.json',
 'pegasus_paraphrase/spiece.model',
 'pegasus_paraphrase/added_tokens.json')

In [6]:
# Load saved model and tokenizer

model=PegasusForConditionalGeneration.from_pretrained('pegasus_paraphrase')
tokenizer=PegasusTokenizer.from_pretrained('pegasus_paraphrase')

Build predictive system

In [7]:
def get_response(input_text, num_retrurn_sequences=5, num_beams=5):
  batch = tokenizer([text], padding =True, truncation=True, max_length=60, return_tensors='pt')
  translated=model.generate(**batch, max_length=60, num_beams=num_beams, num_return_sequences=num_retrurn_sequences, temperature=1.5)
  tgt_text=tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [8]:
num_beams=5
num_return_sequences=5
text='Machine learning is important in Data science'
get_response(text, num_beams, num_return_sequences)

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2691: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Keyword arguments {'trauncation': True} not recognized.
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


['Data science uses machine learning.',
 'Machine learning is important.',
 'Machine learning is important in data science.',
 'Data science uses machine learning to solve problems.',
 'Machine learning is important for data science.']